In [1]:
TOTAL_CHAR = 2**16


# derivative first half (log P(s) / alpha)
import numpy as np

import sys
import pickle
import math
import numpy as np
import pdb

np.set_printoptions(suppress=True)
np.set_printoptions(linewidth=400)
# n = 4   # number of letters as context
MAX_N = 4


class Blending_with_linear_param:
    max_f = 0

    def __init__(self, max_f):
        self.max_f = max_f
        return None

    def derivative_of_P_to_alpha(self, alpha_matrix, beta_matrix, model, prefix, x):

        n = len(prefix)
        alpha, beta = self.compute_alpha_beta(alpha_matrix, beta_matrix, prefix, model)
        if prefix in model:
            ms = sum(model[prefix].values())
            us = len(model[prefix])
            if x in model[prefix]:
                ms_x = model[prefix][x]
            else:
                ms_x = 0
        else:
            ms = 0
            us = 0
            ms_x = 0

        eq1 = (beta-ms_x) / (ms+alpha)**2

        if len(prefix) == 0:
            eq2 = (ms-us*beta) / (ms+alpha)**2 / TOTAL_CHAR
            eq3 = 0
        else:
            next_prefix = prefix[1:]
            eq2 = (ms-us*beta) / (ms+alpha)**2 * self.blend(alpha_matrix, beta_matrix, next_prefix, x, model)
            eq3 = (alpha + us*beta) / (ms+alpha) * self.derivative_of_P_to_beta(alpha_matrix, beta_matrix, model, next_prefix, x)

        return eq1 + eq2 + eq3

    def derivative_of_P_to_beta(self, alpha_matrix, beta_matrix, model, prefix, x):

        n = len(prefix)
        alpha, beta = self.compute_alpha_beta(alpha_matrix, beta_matrix, prefix, model)
        if prefix in model:
            ms = sum(model[prefix].values())
            us = len(model[prefix])
            if x in model[prefix]:
                ms_x = model[prefix][x]
            else:
                ms_x = 0
        else:
            ms = 0
            us = 0
            ms_x = 0

        eq1 = -1 / (ms+alpha)
        if len(prefix) == 0:
            eq2 = (ms-us*beta) / (ms+alpha)**2 / TOTAL_CHAR
            eq3 = 0
        else:
            next_prefix = prefix[1:]
            eq2 = us / (ms+alpha) * self.blend(alpha_matrix, beta_matrix, next_prefix, x, model, n)
            eq3 = (alpha + us*beta) / (ms+alpha) * self.derivative_of_P_to_alpha(alpha_matrix, beta_matrix, model, next_prefix, x)

        return eq1 + eq2 + eq3


    def update_param(self, alpha_matrix, beta_matrix, prefix, x, model):
        learning_rate = 0.003
        n = len(prefix)
        # add a computation for alpha beta
        param_derivative_mapping = [1, len(set(prefix))]


        for ind, alpha_derivative in zip(range(len(alpha_matrix[n])), param_derivative_mapping):
            derivative_alpha = 1 / self.blend(alpha_matrix, beta_matrix, prefix, x, model, n) * self.derivative_of_P_to_alpha(alpha_matrix, beta_matrix, model, prefix, x) * alpha_derivative
            alpha_matrix[n][ind] += learning_rate * derivative_alpha

        for ind, beta_derivative in zip(range(len(beta_matrix[n])), param_derivative_mapping):
            derivative_beta = 1 / self.blend(alpha_matrix, beta_matrix, prefix, x, model, n) * self.derivative_of_P_to_beta(alpha_matrix, beta_matrix, model, prefix, x) * beta_derivative
            beta_matrix[n][ind] += learning_rate * derivative_beta

        # restrict param for beta
        beta_matrix[n][0] = max(0, min(1, beta_matrix[n][0]))

        if beta_matrix[n][0] + self.max_f * beta_matrix[n][1] > 1:
            beta_matrix[n][1] = (1 - beta_matrix[n][0]) / self.max_f
        elif beta_matrix[n][0] + self.max_f * beta_matrix[n][1] < 0:
            beta_matrix[n][1] = -beta_matrix[n][0] / self.max_f

        max_beta = max(beta_matrix[n][0], self.max_f*beta_matrix[n][1])
        # restrict param for alpha
        alpha_matrix[n][0] = max(alpha_matrix[n][0], -max_beta)

        if alpha_matrix[n][0] + self.max_f * alpha_matrix[n][1] < 0:
            alpha_matrix[n][1] = -alpha_matrix[n][0] / self.max_f



    def compute_alpha_beta(self, alpha_matrix, beta_matrix, prefix, model):
        n = len(prefix)
        if prefix in model:
            f = len(model[prefix])
        else:
            f = 0

        if f > self.max_f:
            f = self.max_f

        param_derivative_mapping = [1, f]
        # alpha1 = np.dot(alpha_matrix[n], param_derivative_mapping)
        # beta1 = np.dot(beta_matrix[n], param_derivative_mapping)

        alpha = alpha_matrix[n][0] + f*  alpha_matrix[n][1]
        beta = beta_matrix[n][0] + f*  beta_matrix[n][1]

        #beta = max(0, min(1, beta))
        # alpha = max(-beta, alpha)


        return (alpha, beta)

    def build_model_with_train(self, data, alpha_matrix, beta_matrix, word_file, n):
        """
        Build n-gram model of the words in  words_lang.txt
        """
        n_grams = {}
        # read in all n+1 grams
        n_plus_1_gram_counts = {}
        for word in data:
            # update n-gram
            if not word or word[0] == "#" or " " in word:
                continue
            word = "^" * n + word.strip() + "$"
            for i in range(len(word) - n):
                n_plus_1_gram = word[i:i + n + 1]

                for N in range(n, -1, -1):
                    x = n_plus_1_gram[-1]
                    next_n = n_plus_1_gram[:-1]

                    if next_n not in n_grams:
                        n_grams[next_n] = {}

                    if x not in n_grams[next_n]:
                        n_grams[next_n][x] = 1
                    else:
                        n_grams[next_n][x] += 1

                    n_plus_1_gram = n_plus_1_gram[1:]

            # update alpha and beta
            for i in range(len(word) - n):
                n_plus_1_gram = word[i:i + n + 1]

                for N in range(n, -1, -1):
                    x = n_plus_1_gram[-1]
                    next_n = n_plus_1_gram[:-1]

                    self.update_param(alpha_matrix, beta_matrix, next_n, x, n_grams)

                    n_plus_1_gram = n_plus_1_gram[1:]

        return n_grams

    def blend(self, alpha_matrix, beta_matrix, prefix, x, model, n=4):
        prob = 0

        # compute alpha and beta
        alpha, beta = self.compute_alpha_beta(alpha_matrix, beta_matrix, prefix, model)
        possible_x = TOTAL_CHAR

        # first half of algorithm
        if prefix in model:
            base = (len(model[prefix])*beta + alpha) / (sum(model[prefix].values()) + alpha)
            if x in model[prefix]:
                prob += (model[prefix][x] - beta) / (sum(model[prefix].values()) + alpha)
        else:
            base = (0 + alpha) / (0 + alpha)

        if len(prefix) == 0:
            prob += base / possible_x
        else:
            prob += base * self.blend(alpha_matrix, beta_matrix, prefix[1:], x, model, n)

        return prob


    def word_prob_blend(self, alpha_matrix, beta_matrix, word, model, n=4):
        word = "^" * n + word.strip() + "$"
        pos = n  # char after n ^
        log_likelihood = 0
        # print ("   ", end="")
        while pos < len(word):
            prefix = word[pos - n:pos]
            prob = self.blend(alpha_matrix, beta_matrix, prefix, word[pos], model, n)
            # print(prefix, word[pos], prob)
            if prob:
                log_likelihood += math.log (prob)
            pos += 1


        return log_likelihood / (len(word) - 1)  # didn't guess "^"



In [2]:
# trainer with 2d matrix for d and f

class Blending_with_2d_matrix(Blending_with_linear_param):
    def init(self, max_f):
        super(max_f)
        return None

    def update_param(self, alpha_matrix, beta_matrix, prefix, x, model):
        learning_rate = 0.003
        d = len(prefix)
        n = d
        if prefix in model:
            f = len(model[prefix])
        else:
            f = 0

        if f > self.max_f:
            f = self.max_f

        derivative_beta = 1 / self.blend(alpha_matrix, beta_matrix, prefix, x, model, n) * self.derivative_of_P_to_beta(alpha_matrix, beta_matrix, model, prefix, x)
        beta_matrix[d][f] += learning_rate * derivative_beta

        beta_matrix[d][f] = max(0, min(1, beta_matrix[d][f]))

        derivative_alpha = 1 / self.blend(alpha_matrix, beta_matrix, prefix, x, model, n) * self.derivative_of_P_to_alpha(alpha_matrix, beta_matrix, model, prefix, x)
        alpha_matrix[d][f] += learning_rate * derivative_alpha

        alpha_matrix[d][f] = max(-beta_matrix[d][f], alpha_matrix[d][f])



    def compute_alpha_beta(self, alpha_matrix, beta_matrix, prefix, model):
        d = len(prefix)

        if prefix in model:
            f = len(model[prefix])
        else:
            f = 0

        if f > self.max_f:
            f = self.max_f

        return (alpha_matrix[d][f], beta_matrix[d][f])

In [ ]:
# test build model and train
MAX_F = 10
n = 5

# load data
data = []
with open ("words_manual_en.txt", "r") as f :
    for word in f:
        data.append(word)

alpha_matrix = [[0.5, 0] for _ in range(n+1)]
beta_matrix = [[0.75, 0] for _ in range(n+1)]

trainer_1 = Blending_with_linear_param(MAX_F)
n_gram_model_1 = trainer_1.build_model_with_train(alpha_matrix, beta_matrix, "words_manual_en.txt", n)

# verify
cand = [chr(i) for i in range(ord('A'), ord('Z') + 1)] + [chr(i) for i in range(ord('a'), ord('z') + 1)] + ["$"]

# alpha_matrix = [[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
# beta_matrix = [[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
def verify(trainer, prefix, model):
    s = 0
    for c in cand:
        prob = trainer.blend(alpha_matrix, beta_matrix, prefix, c, model, n)
        s += prob

    return s

print(verify(trainer_1, "ralv", n_gram_model_1))
print(verify(trainer_1, "^All", n_gram_model_1))
print(verify(trainer_1, "Trum", n_gram_model_1))
print(verify(trainer_1, "Russ", n_gram_model_1))
print(verify(trainer_1, "Koch", n_gram_model_1))
print(verify(trainer_1, "Rack", n_gram_model_1))

0.9999999999999999
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0


In [ ]:
# test build model and train
MAX_F = 10
n = 5

# load data
data = []
with open ("words_manual_en.txt", "r") as f :
    for word in f:
        data.append(word)

alpha_matrix = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
beta_matrix = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

trainer_2 = Blending_with_2d_matrix(MAX_F)
n_gram_model_2 = trainer_2.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)

# verify
cand = [chr(i) for i in range(ord('A'), ord('Z') + 1)] + [chr(i) for i in range(ord('a'), ord('z') + 1)] + ["$"]

def verify(trainer, prefix, model, n=n):
    s = 0
    for c in cand:
        prob = trainer.blend(alpha_matrix, beta_matrix, prefix, c, model)

        s += prob

    return s

print(verify(trainer_2, "ralv", n_gram_model_2))
print(verify(trainer_2, "^All", n_gram_model_2))
print(verify(trainer_2, "Trum", n_gram_model_2))
print(verify(trainer_2, "Russ", n_gram_model_2))
print(verify(trainer_2, "Koch", n_gram_model_2))
print(verify(trainer_2, "Rack", n_gram_model_2))

1.0
0.999999999744496
1.0
1.0
1.0
1.0


In [ ]:
beta_matrix

[[0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.6850448508074591, 0.75, 0.75, 0.75, 0],
 [0.75,
  0.38409422131037535,
  0.4364834595076074,
  0.44368669244751446,
  0.49456958006947976,
  0.4758418591768404,
  0.5432620677200376,
  0.5889374387396005,
  0.5721489144783766,
  0.6248700104831192,
  0.00022023063195687254],
 [0.75,
  0,
  0,
  0,
  0.07090271169346975,
  0.2580670172289869,
  0.3898318945902937,
  0.4647159029352439,
  0.5597668408931874,
  0.567961018414197,
  0.3283024740495148],
 [0.75,
  0,
  0.0018261506595701968,
  0.18288404908030825,
  0.38100256242195457,
  0.5123989433527012,
  0.6013934670050458,
  0.6690310715422526,
  0.6850879957947016,
  0.7127280704797957,
  0.5436575850432567],
 [0.75,
  0,
  0.2504879866991498,
  0.4465826846203506,
  0.510607161145451,
  0.5728522828300227,
  0.6229852543617678,
  0.677999897204668,
  0.6848499927185444,
  0.7154580369806254,
  0.5376696016640183],
 [0.75,
  0,
  0.39500743908400676,
  0.516119825427136,
  0.5309930287467219,


Compare training and non training performance

In [ ]:
# test build model and train for linear equation pram method
MAX_F = 10
n = 5

# load data
data = []
with open ("words_manual_en.txt", "r") as f :
    for word in f:
        data.append(word)

alpha_matrix = [[0.5, 0] for _ in range(n+1)]
beta_matrix = [[0.75, 0] for _ in range(n+1)]

trainer_1 = Blending_with_linear_param(MAX_F)
n_gram_model_1 = trainer_1.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)

alpha_matrix_untrained = [[0.5, 0] for _ in range(n+1)]
beta_matrix_untrained = [[0.75, 0] for _ in range(n+1)]

iter = 1000
count = 0
with open ("words_manual_en.txt", "r") as f :
    total_un = 0
    total = 0
    for word in f:
        word = word.strip()
        untrained_score = trainer_1.word_prob_blend (alpha_matrix_untrained, beta_matrix_untrained, word, n_gram_model_1, n)
        trained_score = trainer_1.word_prob_blend (alpha_matrix, beta_matrix, word, n_gram_model_1, n)

        total_un += untrained_score
        total += trained_score

        count += 1
        if count == iter:
            break

    print(f"avg untrained: {-total_un/iter}, avg trained: {-total/iter}")

avg untrained: 0.41165258236321056, avg trained: 0.3019432506125171


In [ ]:
# test build model and train for matrix method
import random

data_amount = 10000
count = 0

# load data
data = []
with open ("../checked_words.txt", "r") as f :
    for word in f:
        if not word or word[0] == "#" or " " in word:
            continue

        data.append(word)

random.shuffle(data)
data = data[:data_amount]

MAX_F = 10
n = 5
alpha_matrix = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
beta_matrix = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

trainer_2 = Blending_with_2d_matrix(MAX_F)
n_gram_model_2 = trainer_2.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)

alpha_matrix_untrained = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
beta_matrix_untrained = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]


total_un = 0
total = 0
for word in data:
    word = word.strip()
    untrained_score = trainer_2.word_prob_blend (alpha_matrix_untrained, beta_matrix_untrained, word, n_gram_model_2, n)
    trained_score = trainer_2.word_prob_blend (alpha_matrix, beta_matrix, word, n_gram_model_2, n)

    total_un += untrained_score
    total += trained_score

    count += 1
    if count == data_amount:
        break

print(f"avg untrained: {-total_un/data_amount}, avg trained: {-total/data_amount}")

avg untrained: 1.0146175802848714, avg trained: 0.7400299799082519


Compare the effect of input sequence

In [ ]:
# test input sequence for matrix method
import random
MAX_F = 10
n = 5
data_amount = 10000

for i in range(10):
    # load data
    data = []
    with open ("./checked_words.txt", "r") as f :
        for word in f:
            if not word or word[0] == "#" or " " in word:
                continue

            data.append(word)

    random.shuffle(data)
    data = data[:data_amount]

    alpha_matrix = [[0.5, 0] for _ in range(n+1)]
    beta_matrix = [[0.75, 0] for _ in range(n+1)]

    trainer_linear = Blending_with_linear_param(MAX_F)
    n_gram_model_linear = trainer_linear.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)

    iter = data_amount
    count = 0

    total_un = 0
    total = 0
    for word in data:
        word = word.strip()
        trained_score = trainer_linear.word_prob_blend (alpha_matrix, beta_matrix, word, n_gram_model_linear, n)

        total += trained_score

        count += 1
        if count == iter:
            break

    print(f"iter: {i}, avg trained: {-total/iter}")

iter: 0, avg trained: 0.740998336636894
iter: 1, avg trained: 0.7392492446974981
iter: 2, avg trained: 0.7380248301671555
iter: 3, avg trained: 0.7403750717504713
iter: 4, avg trained: 0.7394730937184694
iter: 5, avg trained: 0.7382880159746086
iter: 6, avg trained: 0.739922362326643
iter: 7, avg trained: 0.7409449485654295
iter: 8, avg trained: 0.7418387884424199
iter: 9, avg trained: 0.740000657742856


In [ ]:
# test input sequence for matrix method
import random
MAX_F = 10
n = 5

data_amount = 10000

for i in range(10):
    # load data
    data = []
    with open ("./checked_words.txt", "r") as f :
        for word in f:
            data.append(word)

    random.shuffle(data)

    data = data[:data_amount]

    alpha_matrix = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    trainer_matrix = Blending_with_2d_matrix(MAX_F)
    n_gram_model_matrix = trainer_matrix.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)

    iter = data_amount
    count = 0

    total_un = 0
    total = 0
    for word in data:
        word = word.strip()
        trained_score = trainer_matrix.word_prob_blend (alpha_matrix, beta_matrix, word, n_gram_model_matrix, n)

        total += trained_score

        count += 1
        if count == iter:
            break

    print(f"iter: {i}, avg trained: {-total/iter}")

iter: 0, avg trained: 0.7549226877242979
iter: 1, avg trained: 0.7446704753398885
iter: 2, avg trained: 0.7474707032271138
iter: 3, avg trained: 0.7461749414704258
iter: 4, avg trained: 0.748631143059923
iter: 5, avg trained: 0.7501941398688521
iter: 6, avg trained: 0.748961446381134
iter: 7, avg trained: 0.7453006593200538
iter: 8, avg trained: 0.743932771226342
iter: 9, avg trained: 0.7492689048660606


Compare Performance on different data amount

In [ ]:
#@title 1000 words

import random
MAX_F = 8
n = 5
data_amount = 1000

for i in range(3):
    # load data
    data = []
    with open ("./checked_words.txt", "r") as f :
        for word in f:
            if not word or word[0] == "#" or " " in word:
                continue

            data.append(word)

    random.shuffle(data)
    data = data[:data_amount]

    alpha_matrix_linear = [[0.5, 0] for _ in range(n+1)]
    beta_matrix_linear = [[0.75, 0] for _ in range(n+1)]

    trainer_linear = Blending_with_linear_param(MAX_F)
    n_gram_model_linear = trainer_linear.build_model_with_train(data, alpha_matrix_linear, beta_matrix_linear, "words_manual_en.txt", n)

    alpha_matrix = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    trainer_matrix = Blending_with_2d_matrix(MAX_F)
    n_gram_model_matrix = trainer_matrix.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)

    alpha_matrix_untrained = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix_untrained = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    iter = data_amount
    count = 0

    total_linear = 0
    total_matrix = 0
    total_untrained = 0
    for word in data:
        word = word.strip()
        trained_score_linear = trainer_linear.word_prob_blend (alpha_matrix_linear, beta_matrix_linear, word, n_gram_model_linear, n)
        trained_score_matrix = trainer_matrix.word_prob_blend (alpha_matrix, beta_matrix, word, n_gram_model_matrix, n)
        score_untrained = trainer_matrix.word_prob_blend (alpha_matrix_untrained, beta_matrix_untrained, word, n_gram_model_matrix, n)

        total_linear += trained_score_linear
        total_matrix += trained_score_matrix
        total_untrained += score_untrained

        count += 1
        if count == iter:
            break

    print(f"iter: {i}, avg untrained: {-total_untrained/iter}, avg trained linear: {-total_linear/iter}, avg trained matrix: {-total_matrix/iter}")

iter: 0, avg untrained: 0.8402053405709718, avg trained linear: 0.5074749763661885, avg trained matrix: 0.5138055600933474
iter: 1, avg untrained: 0.8446068929304558, avg trained linear: 0.5083719386733805, avg trained matrix: 0.515578565486036
iter: 2, avg untrained: 0.8495733317798241, avg trained linear: 0.5099657978540284, avg trained matrix: 0.515855327464602


In [ ]:
#@title display
print("alpha: ")
alpha_matrix_display = np.array(np.around(np.array(alpha_matrix),6))

print(alpha_matrix_display)

print("beta: ")
beta_matrix_display = np.array(np.around(np.array(beta_matrix),6))

print(beta_matrix_display)

alpha: 
[[ 0.5       0.5       0.5       0.5       0.5       0.5       0.5       0.5       0.477942]
 [ 0.5       0.433724  0.463315  0.480665  0.472725  0.484867  0.487819  0.482833  0.428497]
 [ 0.5       0.        0.028889  0.252987  0.347411  0.384376  0.429591  0.446644  0.355195]
 [ 0.5       0.        0.        0.        0.244636  0.356425  0.411977  0.450191  0.415046]
 [ 0.5       0.       -0.000119  0.185681  0.326307  0.386342  0.42655   0.451353  0.429473]
 [ 0.5       0.        0.102556  0.331773  0.38668   0.420712  0.442723  0.455019  0.429501]]
beta: 
[[0.75     0.75     0.75     0.75     0.75     0.75     0.75     0.75     0.098965]
 [0.75     0.536362 0.565355 0.606157 0.579237 0.598968 0.631124 0.607074 0.00079 ]
 [0.75     0.       0.       0.       0.       0.008839 0.128911 0.20776  0.      ]
 [0.75     0.       0.       0.       0.       0.       0.216083 0.399318 0.289807]
 [0.75     0.       0.000119 0.203245 0.484738 0.596683 0.622932 0.653657 0.484271]
 [0.75

In [ ]:
#@title 5000 words

import random
MAX_F = 8
n = 5
data_amount = 5000

for i in range(3):
    # load data
    data = []
    with open ("./checked_words.txt", "r") as f :
        for word in f:
            if not word or word[0] == "#" or " " in word:
                continue

            data.append(word)

    random.shuffle(data)
    data = data[:data_amount]

    alpha_matrix_linear = [[0.5, 0] for _ in range(n+1)]
    beta_matrix_linear = [[0.75, 0] for _ in range(n+1)]

    trainer_linear = Blending_with_linear_param(MAX_F)
    n_gram_model_linear = trainer_linear.build_model_with_train(data, alpha_matrix_linear, beta_matrix_linear, "words_manual_en.txt", n)

    alpha_matrix = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    trainer_matrix = Blending_with_2d_matrix(MAX_F)
    n_gram_model_matrix = trainer_matrix.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)

    alpha_matrix_untrained = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix_untrained = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    iter = data_amount
    count = 0

    total_linear = 0
    total_matrix = 0
    total_untrained = 0
    for word in data:
        word = word.strip()
        trained_score_linear = trainer_linear.word_prob_blend (alpha_matrix_linear, beta_matrix_linear, word, n_gram_model_linear, n)
        trained_score_matrix = trainer_matrix.word_prob_blend (alpha_matrix, beta_matrix, word, n_gram_model_matrix, n)
        score_untrained = trainer_matrix.word_prob_blend (alpha_matrix_untrained, beta_matrix_untrained, word, n_gram_model_matrix, n)

        total_linear += trained_score_linear
        total_matrix += trained_score_matrix
        total_untrained += score_untrained

        count += 1
        if count == iter:
            break

    print(f"iter: {i}, avg untrained: {-total_untrained/iter}, avg trained linear: {-total_linear/iter}, avg trained matrix: {-total_matrix/iter}")

iter: 0, avg untrained: 0.9693999320088026, avg trained linear: 0.6639650325067626, avg trained matrix: 0.6651876337171544
iter: 1, avg untrained: 0.9660362334981374, avg trained linear: 0.6628228885157216, avg trained matrix: 0.6629567397292503
iter: 2, avg untrained: 0.9668027520628751, avg trained linear: 0.6646286169338222, avg trained matrix: 0.665901792696938


In [ ]:
#@title display
print("alpha: ")
alpha_matrix_display = np.array(np.around(np.array(alpha_matrix),6))

print(alpha_matrix_display)

print("beta: ")
beta_matrix_display = np.array(np.around(np.array(beta_matrix),6))

print(beta_matrix_display)

alpha: 
[[0.5      0.5      0.5      0.5      0.5      0.5      0.5      0.5      0.473042]
 [0.5      0.422741 0.445967 0.476002 0.476926 0.483987 0.48104  0.489769 0.415594]
 [0.5      0.       0.       0.13631  0.263342 0.321436 0.368351 0.413834 0.202088]
 [0.5      0.       0.       0.       0.000419 0.000197 0.157025 0.295586 0.089942]
 [0.5      0.       0.       0.       0.000023 0.156946 0.280989 0.338602 0.276618]
 [0.5      0.       0.       0.000193 0.187012 0.297198 0.345003 0.380123 0.357913]]
beta: 
[[0.75     0.75     0.75     0.75     0.75     0.75     0.75     0.75     0.      ]
 [0.75     0.497352 0.523135 0.598617 0.580661 0.607304 0.59326  0.61551  0.003667]
 [0.75     0.       0.       0.       0.       0.       0.       0.000741 0.008904]
 [0.75     0.       0.       0.       0.000391 0.       0.       0.       0.000371]
 [0.75     0.       0.       0.       0.       0.       0.002752 0.196581 0.012236]
 [0.75     0.       0.       0.       0.176436 0.330746 0.39

In [ ]:
#@title 10000 words

import random
MAX_F = 8
n = 4
data_amount = 10000

for i in range(2):
    n+=1
    # load data
    data = []
    with open ("./checked_words.txt", "r") as f :
        for word in f:
            if not word or word[0] == "#" or " " in word:
                continue

            data.append(word)

    random.shuffle(data)
    data = data[:data_amount]

    alpha_matrix_linear = [[0.5, 0] for _ in range(n+1)]
    beta_matrix_linear = [[0.75, 0] for _ in range(n+1)]

    trainer_linear = Blending_with_linear_param(MAX_F)
    n_gram_model_linear = trainer_linear.build_model_with_train(data, alpha_matrix_linear, beta_matrix_linear, "words_manual_en.txt", n)

    alpha_matrix = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    trainer_matrix = Blending_with_2d_matrix(MAX_F)
    n_gram_model_matrix = trainer_matrix.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)

    alpha_matrix_untrained = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix_untrained = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    iter = data_amount
    count = 0

    total_linear = 0
    total_matrix = 0
    total_untrained = 0
    for word in data:
        word = word.strip()
        trained_score_linear = trainer_linear.word_prob_blend (alpha_matrix_linear, beta_matrix_linear, word, n_gram_model_linear, n)
        trained_score_matrix = trainer_matrix.word_prob_blend (alpha_matrix, beta_matrix, word, n_gram_model_matrix, n)
        score_untrained = trainer_matrix.word_prob_blend (alpha_matrix_untrained, beta_matrix_untrained, word, n_gram_model_matrix, n)

        total_linear += trained_score_linear
        total_matrix += trained_score_matrix
        total_untrained += score_untrained

        count += 1
        if count == iter:
            break

    print(f"iter: {i}, avg untrained: {-total_untrained/iter}, avg trained linear: {-total_linear/iter}, avg trained matrix: {-total_matrix/iter}")

iter: 0, avg untrained: 1.0145477577726594, avg trained linear: 0.7396963256134947, avg trained matrix: 0.7401879083582934
iter: 1, avg untrained: 0.851108915156914, avg trained linear: 0.6285709543294501, avg trained matrix: 0.6290525500223288


In [ ]:
#@title display
print("alpha: ")
alpha_matrix_display = np.array(np.around(np.array(alpha_matrix),6))

print(alpha_matrix_display)

print("beta: ")
beta_matrix_display = np.array(np.around(np.array(beta_matrix),6))

print(beta_matrix_display)

alpha: 
[[ 0.5       0.5       0.5       0.5       0.5       0.5       0.5       0.497158  0.473376]
 [ 0.5       0.42087   0.441385  0.46379   0.481298  0.480161  0.485429  0.4889    0.400198]
 [ 0.5       0.        0.        0.097654  0.23505   0.313973  0.357209  0.395761  0.130881]
 [ 0.5       0.        0.       -0.000271  0.        0.       -0.000024  0.173344 -0.00005 ]
 [ 0.5       0.        0.       -0.000284  0.        0.000157  0.100113  0.237276  0.15784 ]
 [ 0.5       0.        0.000024  0.        0.035136  0.151986  0.272224  0.313389  0.326997]
 [ 0.5       0.        0.000502  0.088199  0.220262  0.29429   0.340362  0.355521  0.36485 ]]
beta: 
[[0.75     0.75     0.75     0.75     0.75     0.75     0.75     0.686549 0.      ]
 [0.75     0.48956  0.490635 0.53078  0.598935 0.584342 0.597283 0.608569 0.015595]
 [0.75     0.       0.       0.000153 0.       0.000222 0.000001 0.000728 0.00662 ]
 [0.75     0.       0.       0.000292 0.       0.       0.000024 0.000842 0.00032

In [ ]:
#@title 50000 words

import random
MAX_F = 8
n = 5
data_amount = 50000

for i in range(1):
    # load data
    data = []
    with open ("./checked_words.txt", "r") as f :
        for word in f:
            if not word or word[0] == "#" or " " in word:
                continue

            data.append(word)

    random.shuffle(data)
    data = data[:data_amount]

    alpha_matrix_linear = [[0.5, 0] for _ in range(n+1)]
    beta_matrix_linear = [[0.75, 0] for _ in range(n+1)]

    trainer_linear = Blending_with_linear_param(MAX_F)
    n_gram_model_linear = trainer_linear.build_model_with_train(data, alpha_matrix_linear, beta_matrix_linear, "words_manual_en.txt", n)

    alpha_matrix = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    trainer_matrix = Blending_with_2d_matrix(MAX_F)
    n_gram_model_matrix = trainer_matrix.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)

    alpha_matrix_untrained = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix_untrained = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    iter = data_amount
    count = 0

    total_linear = 0
    total_matrix = 0
    total_untrained = 0
    for word in data:
        word = word.strip()
        trained_score_linear = trainer_linear.word_prob_blend (alpha_matrix_linear, beta_matrix_linear, word, n_gram_model_linear, n)
        trained_score_matrix = trainer_matrix.word_prob_blend (alpha_matrix, beta_matrix, word, n_gram_model_matrix, n)
        score_untrained = trainer_matrix.word_prob_blend (alpha_matrix_untrained, beta_matrix_untrained, word, n_gram_model_matrix, n)

        total_linear += trained_score_linear
        total_matrix += trained_score_matrix
        total_untrained += score_untrained

        count += 1
        if count == iter:
            break

    print(f"iter: {i}, avg untrained: {-total_untrained/iter}, avg trained linear: {-total_linear/iter}, avg trained matrix: {-total_matrix/iter}")

iter: 0, avg untrained: 1.1112304899030658, avg trained linear: 0.926577522097799, avg trained matrix: 0.9263261016973349


In [ ]:
#@title display
print("alpha: ")
alpha_matrix_display = np.array(np.around(np.array(alpha_matrix),6))

print(alpha_matrix_display)

print("beta: ")
beta_matrix_display = np.array(np.around(np.array(beta_matrix),6))

print(beta_matrix_display)

alpha: 
[[ 0.5       0.5       0.5       0.5       0.5       0.5       0.5       0.497143  0.468532]
 [ 0.5       0.400328  0.434745  0.455257  0.469214  0.471378  0.484841  0.482849  0.392027]
 [ 0.5       0.       -0.000002 -0.00003   0.140007  0.233829  0.306843  0.357974  0.008133]
 [ 0.5       0.        0.000015  0.000028 -0.000055  0.000133 -0.000033 -0.000123  0.00014 ]
 [ 0.5       0.       -0.00012  -0.000242  0.        0.       -0.000066 -0.000255  0.000133]
 [ 0.5       0.       -0.        0.        0.        0.       -0.000001  0.106973  0.12396 ]]
beta: 
[[0.75     0.75     0.75     0.75     0.75     0.75     0.75     0.69187  0.      ]
 [0.75     0.441817 0.452762 0.473427 0.492014 0.525978 0.594012 0.562906 0.006045]
 [0.75     0.       0.000038 0.00003  0.       0.       0.       0.       0.006888]
 [0.75     0.       0.000244 0.000009 0.000309 0.000019 0.00004  0.002564 0.004218]
 [0.75     0.       0.00012  0.000494 0.       0.       0.000066 0.00077  0.001248]
 [0.75

In [ ]:
#@title 100000 words

import random
MAX_F = 8
n = 5
data_amount = 100000

for i in range(1):
    # load data
    data = []
    with open ("./checked_words.txt", "r") as f :
        for word in f:
            if not word or word[0] == "#" or " " in word:
                continue

            data.append(word)

    random.shuffle(data)
    data = data[:data_amount]

    alpha_matrix_linear = [[0.5, 0] for _ in range(n+1)]
    beta_matrix_linear = [[0.75, 0] for _ in range(n+1)]

    trainer_linear = Blending_with_linear_param(MAX_F)
    n_gram_model_linear = trainer_linear.build_model_with_train(data, alpha_matrix_linear, beta_matrix_linear, "words_manual_en.txt", n)

    alpha_matrix = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    trainer_matrix = Blending_with_2d_matrix(MAX_F)
    n_gram_model_matrix = trainer_matrix.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)

    alpha_matrix_untrained = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix_untrained = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    iter = data_amount
    count = 0

    total_linear = 0
    total_matrix = 0
    total_untrained = 0
    for word in data:
        word = word.strip()
        trained_score_linear = trainer_linear.word_prob_blend (alpha_matrix_linear, beta_matrix_linear, word, n_gram_model_linear, n)
        trained_score_matrix = trainer_matrix.word_prob_blend (alpha_matrix, beta_matrix, word, n_gram_model_matrix, n)
        score_untrained = trainer_matrix.word_prob_blend (alpha_matrix_untrained, beta_matrix_untrained, word, n_gram_model_matrix, n)

        total_linear += trained_score_linear
        total_matrix += trained_score_matrix
        total_untrained += score_untrained

        count += 1
        if count == iter:
            break

    print(f"iter: {i}, avg untrained: {-total_untrained/iter}, avg trained linear: {-total_linear/iter}, avg trained matrix: {-total_matrix/iter}")

iter: 0, avg untrained: 1.1494670692868, avg trained linear: 1.0036818669649628, avg trained matrix: 1.0037602505001888


In [ ]:
#@title display
print("alpha: ")
alpha_matrix_display = np.array(np.around(np.array(alpha_matrix),6))

print(alpha_matrix_display)

print("beta: ")
beta_matrix_display = np.array(np.around(np.array(beta_matrix),6))

print(beta_matrix_display)

alpha: 
[[ 0.5       0.5       0.5       0.5       0.5       0.497273  0.5       0.5       0.465175]
 [ 0.5       0.373589  0.441891  0.449966  0.459534  0.470508  0.481272  0.479496  0.395849]
 [ 0.5       0.        0.       -0.000001  0.090554  0.190307  0.264907  0.353419  0.000375]
 [ 0.5       0.        0.       -0.000084  0.        0.000134  0.        0.000027  0.      ]
 [ 0.5       0.        0.        0.        0.000193  0.000141  0.000357 -0.000313 -0.000192]
 [ 0.5       0.        0.        0.000122 -0.000028  0.000013 -0.000678 -0.000122 -0.000104]]
beta: 
[[0.75     0.75     0.75     0.75     0.75     0.694872 0.75     0.75     0.      ]
 [0.75     0.401877 0.467307 0.436558 0.465392 0.495749 0.516013 0.515379 0.000329]
 [0.75     0.       0.       0.000041 0.000186 0.       0.000025 0.000332 0.001536]
 [0.75     0.       0.       0.000545 0.       0.000784 0.       0.000211 0.      ]
 [0.75     0.       0.       0.       0.000773 0.       0.001912 0.001052 0.004344]
 [0.75

Vary max f and n

In [6]:
#@title vary n

import random
MAX_F = 10
n = 4
data_amount = 10000

for i in range(8):
    n+=1
    # load data
    data = []
    with open ("./checked_words.txt", "r") as f :
        for word in f:
            if not word or word[0] == "#" or " " in word:
                continue

            data.append(word)

    random.shuffle(data)
    data = data[:data_amount]

    alpha_matrix_linear = [[0.5, 0] for _ in range(n+1)]
    beta_matrix_linear = [[0.75, 0] for _ in range(n+1)]

    trainer_linear = Blending_with_linear_param(MAX_F)
    n_gram_model_linear = trainer_linear.build_model_with_train(data, alpha_matrix_linear, beta_matrix_linear, "words_manual_en.txt", n)

    alpha_matrix = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    trainer_matrix = Blending_with_2d_matrix(MAX_F)
    n_gram_model_matrix = trainer_matrix.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)

    alpha_matrix_untrained = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix_untrained = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    iter = data_amount
    count = 0

    total_linear = 0
    total_matrix = 0
    total_untrained = 0
    for word in data:
        word = word.strip()
        trained_score_linear = trainer_linear.word_prob_blend (alpha_matrix_linear, beta_matrix_linear, word, n_gram_model_linear, n)
        trained_score_matrix = trainer_matrix.word_prob_blend (alpha_matrix, beta_matrix, word, n_gram_model_matrix, n)
        score_untrained = trainer_matrix.word_prob_blend (alpha_matrix_untrained, beta_matrix_untrained, word, n_gram_model_matrix, n)

        total_linear += trained_score_linear
        total_matrix += trained_score_matrix
        total_untrained += score_untrained

        count += 1
        if count == iter:
            break

    print(f"iter: {i}, avg untrained: {-total_untrained/iter}, avg trained linear: {-total_linear/iter}, avg trained matrix: {-total_matrix/iter}")

iter: 0, avg untrained: 1.012201485018517, avg trained linear: 0.7384294956786646, avg trained matrix: 0.739126202412177
iter: 1, avg untrained: 0.8495146729639187, avg trained linear: 0.6256500821663094, avg trained matrix: 0.6263357450493791
iter: 2, avg untrained: 0.7443254980407424, avg trained linear: 0.574677768889316, avg trained matrix: 0.5753518482574371
iter: 3, avg untrained: 0.6626572377635526, avg trained linear: 0.5332389412003412, avg trained matrix: 0.5335147449909117
iter: 4, avg untrained: 0.6015135922357661, avg trained linear: 0.5012216022354891, avg trained matrix: 0.501616837753367
iter: 5, avg untrained: 0.5518037948930371, avg trained linear: 0.47370959366659254, avg trained matrix: 0.47392188498178384
iter: 6, avg untrained: 0.5109415425439048, avg trained linear: 0.44929842780930984, avg trained matrix: 0.4498370147778219
iter: 7, avg untrained: 0.47731636600477395, avg trained linear: 0.4298003718484721, avg trained matrix: 0.4283618610886603


In [ ]:
#@title train test split

import random
MAX_F = 10
n = 5
train_ratio = 0.8
data_amount = 10000

for i in range(5):
    train_size = int(train_ratio * data_amount)
    # load data
    data = []
    with open ("./checked_words.txt", "r") as f :
        for word in f:
            if not word or word[0] == "#" or " " in word:
                continue

            data.append(word)

    random.shuffle(data)
    train_data = data[:train_size]

    alpha_matrix_linear = [[0.5, 0] for _ in range(n+1)]
    beta_matrix_linear = [[0.75, 0] for _ in range(n+1)]

    trainer_linear = Blending_with_linear_param(MAX_F)
    n_gram_model_linear = trainer_linear.build_model_with_train(train_data, alpha_matrix_linear, beta_matrix_linear, "words_manual_en.txt", n)

    alpha_matrix = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    trainer_matrix = Blending_with_2d_matrix(MAX_F)
    n_gram_model_matrix = trainer_matrix.build_model_with_train(train_data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)

    alpha_matrix_untrained = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix_untrained = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    iter = data_amount*(1-train_ratio)
    count = 0

    total_linear = 0
    total_matrix = 0
    total_untrained = 0
    for word in data[train_size:]:
        word = word.strip()
        trained_score_linear = trainer_linear.word_prob_blend (alpha_matrix_linear, beta_matrix_linear, word, n_gram_model_linear, n)
        trained_score_matrix = trainer_matrix.word_prob_blend (alpha_matrix, beta_matrix, word, n_gram_model_matrix, n)
        score_untrained = trainer_matrix.word_prob_blend (alpha_matrix_untrained, beta_matrix_untrained, word, n_gram_model_matrix, n)

        total_linear += trained_score_linear
        total_matrix += trained_score_matrix
        total_untrained += score_untrained

    print(f"iter: {i}, avg untrained: {-total_untrained/iter}, avg trained linear: {-total_linear/iter}, avg trained matrix: {-total_matrix/iter}")

In [7]:
#@title 5000 words

import random
MAX_F = 8
n = 5
data_amount = 5000

for i in range(3):
    # load data
    data = []
    with open ("./checked_words.txt", "r") as f :
        for word in f:
            if not word or word[0] == "#" or " " in word:
                continue

            data.append(word)

    random.shuffle(data)
    data = data[:data_amount]

    alpha_matrix_linear = [[0.5, 0] for _ in range(n+1)]
    beta_matrix_linear = [[0.75, 0] for _ in range(n+1)]

    trainer_linear = Blending_with_linear_param(MAX_F)
    n_gram_model_linear = trainer_linear.build_model_with_train(data, alpha_matrix_linear, beta_matrix_linear, "words_manual_en.txt", n)

    alpha_matrix = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    trainer_matrix = Blending_with_2d_matrix(MAX_F)
    n_gram_model_matrix = trainer_matrix.build_model_with_train(data, alpha_matrix, beta_matrix, "words_manual_en.txt", n)

    alpha_matrix_untrained = [[0.5 for _ in range(MAX_F+1)] for _ in range(n+1)]
    beta_matrix_untrained = [[0.75 for _ in range(MAX_F+1)] for _ in range(n+1)]

    iter = data_amount
    count = 0

    total_linear = 0
    total_matrix = 0
    total_untrained = 0
    for word in data:
        word = word.strip()
        trained_score_linear = trainer_linear.word_prob_blend (alpha_matrix_linear, beta_matrix_linear, word, n_gram_model_linear, n)
        trained_score_matrix = trainer_matrix.word_prob_blend (alpha_matrix, beta_matrix, word, n_gram_model_matrix, n)
        score_untrained = trainer_matrix.word_prob_blend (alpha_matrix_untrained, beta_matrix_untrained, word, n_gram_model_matrix, n)

        total_linear += trained_score_linear
        total_matrix += trained_score_matrix
        total_untrained += score_untrained

        count += 1
        if count == iter:
            break

    print(f"iter: {i}, avg untrained: {-total_untrained/iter}, avg trained linear: {-total_linear/iter}, avg trained matrix: {-total_matrix/iter}")

iter: 0, avg untrained: 0.9670477230515387, avg trained linear: 0.6621703969303782, avg trained matrix: 0.6627072837052418
iter: 1, avg untrained: 0.9690150804140151, avg trained linear: 0.6631114039560725, avg trained matrix: 0.6645192464071084
iter: 2, avg untrained: 0.9682885439891415, avg trained linear: 0.6630247734612777, avg trained matrix: 0.6642708293790092


In [9]:
#@title display
print("alpha: ")
alpha_matrix_display = np.array(np.around(np.array(alpha_matrix_linear),8))

print(alpha_matrix_display)

print("beta: ")
beta_matrix_display = np.array(np.around(np.array(beta_matrix_linear),8))

print(beta_matrix_display)

alpha: 
[[ 0.47353782  0.        ]
 [ 0.20684438 -0.02577513]
 [-0.00119564  0.00024771]
 [ 0.00000418  0.00009018]
 [ 0.00017365  0.00052105]
 [ 0.          0.        ]]
beta: 
[[0.         0.        ]
 [0.00076661 0.00076661]
 [0.00467083 0.00916044]
 [0.00072476 0.00194751]
 [0.00020572 0.00061716]
 [0.         0.        ]]


In [10]:
#@title display
print("alpha: ")
alpha_matrix_display = np.array(np.around(np.array(alpha_matrix),8))

print(alpha_matrix_display)

print("beta: ")
beta_matrix_display = np.array(np.around(np.array(beta_matrix),8))

print(beta_matrix_display)

alpha: 
[[ 0.5         0.5         0.5         0.5         0.5         0.5         0.5         0.5         0.47353776]
 [ 0.5         0.43326074  0.45266128  0.47152618  0.4804843   0.48724664  0.48779497  0.48566135  0.40696124]
 [ 0.5         0.          0.          0.14755361  0.25419682  0.32894775  0.3694433   0.40575499  0.20929432]
 [ 0.5         0.          0.         -0.00005069  0.         -0.00013946  0.17928492  0.30040829  0.10880048]
 [ 0.5         0.          0.          0.          0.00064773  0.13726075  0.26679744  0.33140153  0.29766416]
 [ 0.5         0.          0.         -0.00035411  0.1964305   0.29180837  0.35830671  0.38057038  0.37050732]]
beta: 
[[0.75       0.75       0.75       0.75       0.75       0.75       0.75       0.75       0.        ]
 [0.75       0.52848669 0.53129008 0.56782384 0.59052892 0.61829175 0.63136607 0.61760532 0.00076474]
 [0.75       0.         0.         0.         0.         0.00052375 0.         0.         0.00730998]
 [0.75      